In [1]:
%matplotlib qt
import matplotlib.pyplot as plt
import hyperspy.api as hs
import pyxem as pxm
import numpy as np

from pathlib import Path
from matplotlib.colors import SymLogNorm
from skimage.measure import label

from matplotlib.colors import to_rgba
from matplotlib.colors import LinearSegmentedColormap

color_names = ['linen', 'darkorange', 'dodgerblue', 'forestgreen', 'red']
colors = [to_rgba(c) for c in color_names]

cmap = LinearSegmentedColormap.from_list('gt_cmap', colors, N=len(color_names))

In [4]:
datapath = Path(r'C:\Users\emilc\OneDrive - NTNU\NORTEM\Data\2021_10_06_2xxx_24h_250C\Preprocessed_data\SPED_600x600x12_10x10_4p63x4p63_1deg_100Hz_CL12cm_NBD_alpha5_spot1p3_preprocessed.hspy')
signal = hs.load(datapath, lazy=True)

In [6]:
print(f'{int(signal.estimate_elbow_position())}')

5


In [64]:
if False:
    datapath = Path(r'SPED_600x600x12_10x10_4p63x4p63_1deg_100Hz_CL12cm_NBD_alpha5_spot1p3_preprocessed_masked.hspy')
    signal = hs.load(datapath, lazy=True)
    signal.plot_explained_variance_ratio()
    signal.blind_source_separation(6)
    signal.plot_bss_results()
else:
    #14, 15, 16, 17
    datapath = Path(r'C:\Users\emilc\OneDrive - NTNU\NORTEM\Data\2021_10_06_2xxx_24h_250C\Preprocessed_data\NMF\SPED_600x600x12_10x10_4p63x4p63_1deg_100Hz_CL12cm_NBD_alpha5_spot1p3_preprocessed_NMF_15_poissonian_mask.hspy')
    loadings = hs.load(datapath.with_name(f'{datapath.stem}_loadings{datapath.suffix}'))
    factors = hs.load(datapath.with_name(f'{datapath.stem}_factors{datapath.suffix}'))

    loadings.plot()
    factors.plot(norm='symlog')

# Plot learning results

In [65]:
axis_size = 6 #inches
dpi=150
for i, (loading, factor) in enumerate(zip(loadings, factors)):
    fig, axes = plt.subplots(nrows=1, ncols=2, subplot_kw={'xticks': [], 'yticks': []}, figsize=(axis_size*2, axis_size), dpi=dpi)
    axes[0].imshow(loading.data,cmap='viridis_r')
    axes[1].imshow(factor.data, norm=SymLogNorm(0.03),cmap='viridis_r')
    axes[0].annotate(f'Loading {i}', (0.02, 0.98), xycoords='axes fraction', color='w', ha='left', va='top', bbox=dict(facecolor='k', alpha=0.5))
    axes[1].annotate(f'Factor {i}', (0.02, 0.98), xycoords='axes fraction', color='w', ha='left', va='top', bbox=dict(facecolor='k', alpha=0.5))
    plt.tight_layout()
    fig.savefig(f'{datapath.stem}_{i}.png', dpi=dpi)
    plt.close('all')

# Make decomposition map

In [66]:
loadings.plot()
factors.plot(norm='symlog')

14 components

In [53]:
phase_key = {
    'Al': {'components': [], 'threshold': 1},
    'theta`|': {'components': [2, 4, 5, 7, 8, 10, 11, 12], 'threshold': 0.04},
    'theta`': {'components': [13], 'threshold': 0.07},
    'T1': {'components': [1, 3], 'threshold': 0.05},
    'Not indexed': {'components': [], 'threshold': 1}
}

15 components

In [160]:
phase_key = {
    'Al': {'components': [], 'threshold': 1},
    'theta`100': {'components': [2, 4, 5, 7, 8, 10, 11, 13], 'threshold': 0.04},
    'theta`001': {'components': [14], 'threshold': 0.053},
    'T1': {'components': [1, 3], 'threshold': 0.056},
    'Not indexed': {'components': [], 'threshold': 1}
}

## CMAP

In [161]:
phase_map = np.zeros(loadings.axes_manager.signal_shape + (len(phase_key),))
phase_patterns = np.zeros(factors.axes_manager.signal_shape)
ax_size = 3 #inches
fig, axes = plt.subplots(nrows=1, ncols=len(phase_key), figsize=(ax_size*len(phase_key), ax_size), sharex=True, sharey=True, subplot_kw={'xticks': [], 'yticks': []})
for phase_id, (phase, ax) in enumerate(zip(phase_key, axes)):
    if len(phase_key[phase]['components']) > 0:
        for component in phase_key[phase]['components']:
            phase_map[:, :, phase_id] += loadings.inav[component].data/np.nanmax(loadings.inav[component].data)   
        phase_map[:, :, phase_id] /= np.max(phase_map[:, :, phase_id])
        phase_map[:, :, phase_id] = (phase_map[:, :, phase_id]>=phase_key[phase]['threshold'])#np.mean(phase_map[:, :, phase_id]))#-np.std(phase_map[:, :, phase_id]))
    ax.imshow(phase_map[:, :, phase_id])    
    ax.set_title(f'{phase} map')

### Remove edge-on map from other maps

In [162]:
edge_on_phase = 1
fig, axes = plt.subplots(nrows=1, ncols=len(phase_key), figsize=(ax_size*len(phase_key), ax_size), subplot_kw={'xticks': [], 'yticks': []})
for phase_id, (phase, ax) in enumerate(zip(phase_key, axes)):
    if phase_id != edge_on_phase:
        phase_map[:, :, phase_id] -= phase_map[:, :, edge_on_phase]
        phase_map[:, :, phase_id] = phase_map[:, :, phase_id]>0
    ax.imshow(phase_map[:, :, phase_id])
    ax.set_title(f'{phase} map')

### Set overlapping phases to not indexed

In [163]:
not_indexed_id = 4
phase_map[:, :, not_indexed_id] = np.sum(phase_map[:, :, :not_indexed_id], axis=2)>1
fig, axes = plt.subplots(nrows=1, ncols=len(phase_key), figsize=(ax_size*len(phase_key), ax_size), subplot_kw={'xticks': [], 'yticks': []})
for phase_id, (phase, ax) in enumerate(zip(phase_key, axes)):
    if phase_id != not_indexed_id:
        phase_map[:, :, phase_id][phase_map[:, :, not_indexed_id]>0] = 0
    ax.imshow(phase_map[:, :, phase_id])
    ax.set_title(f'{phase} map')

## Set Al to whenever there are no phases or not indexed values

In [164]:
Al_id = 0
phase_map[:, :, Al_id] = ~np.any(phase_map[:, :, Al_id:], axis=2)
fig, axes = plt.subplots(nrows=1, ncols=len(phase_key), figsize=(ax_size*len(phase_key), ax_size), subplot_kw={'xticks': [], 'yticks': []})
for phase_id, (phase, ax) in enumerate(zip(phase_key, axes)):
    ax.imshow(phase_map[:, :, phase_id])
    ax.set_title(f'{phase} map')

## Create phase image

In [165]:
phase_image = np.zeros(phase_map.shape[:2])
for phase_id, phase in enumerate(phase_key):
    phase_image += phase_id*phase_map[:, :, phase_id]
fig = plt.figure(figsize=(ax_size*2, ax_size*2), frameon=False)
ax = fig.add_axes([0, 0, 1, 1], xticks=[], yticks=[], frameon=False)
ax.imshow(phase_image, cmap=cmap, vmax=len(phase_key))
fig.savefig(datapath.with_name(f'{datapath.stem}_phasemap.png'))

In [166]:
phase_image_signal = hs.signals.Signal2D(phase_image)
phase_image_signal.metadata.add_dictionary({'Phase_map_parameters': {'Components': phase_key}, 'Factors': factors, 'Loadings': loadings})
phase_image_signal.save(datapath.with_name(f'{datapath.stem}_NMF_15_phasemap.hdf5'))


Overwrite 'C:\Users\emilc\OneDrive - NTNU\NORTEM\Data\2021_10_06_2xxx_24h_250C\Preprocessed_data\NMF\SPED_600x600x12_10x10_4p63x4p63_1deg_100Hz_CL12cm_NBD_alpha5_spot1p3_preprocessed_NMF_15_poissonian_mask_NMF_15_phasemap.hdf5' (y/n)?
 y


In [167]:
phase_key

{'Al': {'components': [], 'threshold': 1},
 'theta`100': {'components': [2, 4, 5, 7, 8, 10, 11, 13], 'threshold': 0.04},
 'theta`001': {'components': [14], 'threshold': 0.053},
 'T1': {'components': [1, 3], 'threshold': 0.056},
 'Not indexed': {'components': [], 'threshold': 1}}

In [168]:
phase_image_signal.metadata

├── Factors = <ElectronDiffraction2D, title: Decomposition factors of SPED_600x600x12_10x10_4p63x4p63_1deg_100Hz_CL12cm_NBD_alpha5_spot1p3, dimensions: (15|128, 128)>
├── General
│   ├── FileIO
│   │   └── 0
│   │       ├── hyperspy_version = 1.7.0
│   │       ├── io_plugin = hyperspy.io_plugins.hspy
│   │       ├── operation = save
│   │       └── timestamp = 2022-06-01T20:41:50.496517+02:00
│   └── title = 
├── Loadings = <Signal2D, title: Decomposition loadings of SPED_600x600x12_10x10_4p63x4p63_1deg_100Hz_CL12cm_NBD_alpha5_spot1p3, dimensions: (15|512, 512)>
├── Phase_map_parameters
│   └── Components
│       ├── Al
│       │   ├── components = []
│       │   └── threshold = 1
│       ├── Not indexed
│       │   ├── components = []
│       │   └── threshold = 1
│       ├── T1
│       │   ├── components = [1, 3]
│       │   └── threshold = 0.056
│       ├── theta`001
│       │   ├── components = [14]
│       │   └── threshold = 0.053
│       └── theta`100
│           ├── components = [2, 4, 5, 7, 8, 10, 11, 13]
│           └── threshold = 0.04
└── Signal
    └── signal_type =

## Comparison

In [142]:
gt = hs.load(r'C:\Users\emilc\OneDrive - NTNU\NORTEM\Data\2021_10_06_2xxx_24h_250C\Ground_truth_all.hdf5')
gt.plot(cmap=cmap)

C:\Users\emilc\Miniconda3\envs\pyxem0.14.1\lib\site-packages\hyperspy\misc\utils.py:471: VisibleDeprecationWarning: Use of the `binned` attribute in metadata is going to be deprecated in v2.0. Set the `axis.is_binned` attribute instead. 
  warnings.warn(
C:\Users\emilc\Miniconda3\envs\pyxem0.14.1\lib\site-packages\hyperspy\io.py:560: VisibleDeprecationWarning: Loading old file version. The binned attribute has been moved from metadata.Signal to axis.is_binned. Setting this attribute for all signal axes instead.
  warnings.warn('Loading old file version. The binned attribute '


Considering all pixels

In [152]:
difference = gt - phase_image
difference = np.abs(difference.data) > 0

percentage_error = np.count_nonzero(difference) / np.multiply(*gt.axes_manager.signal_shape)
print(f'Percentage error of NMF phase map: {percentage_error:.0%}\nSuccessrate: {1-percentage_error:.0%}')

fig = plt.figure(figsize=(ax_size*2, ax_size*2), frameon=False)
ax = fig.add_axes([0, 0, 1, 1], xticks=[], yticks=[], frameon=False)
ax.imshow(difference, cmap='Greys_r')
fig.savefig(datapath.with_name(f'{datapath.stem}_GT_error.png'))
error_map_signal = hs.signals.Signal2D(difference)
error_map_signal.save(datapath.with_name(f'{datapath.stem}_NMF_15_error.hdf5'))

Percentage error of NMF phase map: 3%
Successrate: 97%


Considering only pixels where ground truth is not Al

In [144]:
difference = gt - phase_image
difference = np.abs(difference.data) > 0

percentage_error = np.count_nonzero(difference[gt.data>0]) / np.count_nonzero(gt.data)
print(f'Percentage error of NMF phase map: {percentage_error:.0%}\nSuccessrate: {1-percentage_error:.0%}')

fig = plt.figure(figsize=(ax_size*2, ax_size*2), frameon=False)
ax = fig.add_axes([0, 0, 1, 1], xticks=[], yticks=[], frameon=False)
ax.imshow(difference, cmap='Greys_r')
fig.savefig(datapath.with_name(f'{datapath.stem}_GT_error_noMatrix.png'))

Percentage error of NMF phase map: 8%
Successrate: 92%


## RGBA

In [147]:
phases = ['T1', 'theta`|', 'theta`']
components = [phase_key[key]['components'] for key in phases]
thresholds = [phase_key[key]['threshold'] for key in phases]

rgba_loadings = np.zeros(loadings.axes_manager.signal_shape + (4,))
rgba_factors = np.zeros(factors.axes_manager.signal_shape + (4,))
for i, (phase, comps) in enumerate(zip(phases, components)):
    factor = np.zeros(factors.axes_manager.signal_shape)
    loading = np.zeros(loadings.axes_manager.signal_shape)
    if len(comps) > 0:
        for c in comps:
            factor += factors.inav[c].data
            loading += loadings.inav[c].data
        factor = factor/np.nanmax(factor)
        loading = loading/np.nanmax(loading)

        rgba_loadings[:, :, i] = loading
        rgba_factors[:, :, i] = factor

        rgba_loadings[:, :, -1] += loading
        rgba_factors[:, :, -1] += factor

        rgba_loadings[:, :, -1] /= np.max(rgba_loadings[:, :, -1])
        rgba_factors[:, :, -1] /= np.max(rgba_factors[:, :, -1])
figure = plt.figure(figsize=(6, 6), frameon=True)
ax = figure.add_axes([0, 0, 1, 1], xticks=[], yticks=[], frameon=False)
ax.imshow(rgba_loadings)
subax = figure.add_axes([0.75, 0.75, 0.25, 0.25], xticks=[], yticks=[], frameon=True)
subax.imshow(rgba_factors)
figure.savefig(f'{datapath.stem}_decomposition_RGBA.png')